## 1. Import libraries and prepare our data.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files


from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans


import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
import xlrd

In [3]:
CLIENT_ID = "PRJWUKQILCPYUNAKTVZ32C4OVZ0M0BFLEO0FPBLLIA02CT1H" # your Foursquare ID
CLIENT_SECRET = 'AMYGYHZMISWJEKHGICR05SJHKBSCZDZJODZ0QPI2ENUHMYE1' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version


In [4]:
df_t = pd.read_excel(r"C:\Users\55016\Desktop\Toronto.xlsx")
df_t.drop(df_t.index[df_t['Borough']=='Not assigned'].values,inplace=True)
df_t.head()

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


In [5]:
df_t.replace({'/': ','}, regex=True,inplace=True)

dft = df_t.groupby(["Postal code","Borough"],as_index=False).sum()

dft.head()

,Postal code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern , Rouge"
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek"
2,M1E,Scarborough,"Guildwood , Morningside , West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_t['Borough'].unique()),
        dft.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


## 2. Let's wrangle and clean our data.

In [7]:
## It seems a bit too long to wait for the response from google, so I just use the csv file provided.
df_geo = pd.read_csv(r"C:\Users\55016\Desktop\Geospatial_Coordinates.csv")

# Let's merge it into our dataset. It's easy for that they are sorted with the same method.
df = pd.concat([dft,df_geo],axis=1)
df.head()

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",M1B,43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",M1C,43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [8]:
# It seems good. We just require to drop the second "Postal code" column.
df.drop("Postal Code",axis=1,inplace=True)

In [9]:
df.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [10]:
## choose the data we would use
df_s = df[df["Borough"]=="Scarborough"].reset_index(drop=True)
df_s.head()

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [12]:
## get the top 100 venues within a radius of 500 metres.
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Scar_venues = getNearbyVenues(names=df_s['Neighborhood'],
                                   latitudes=df_s['Latitude'],
                                   longitudes=df_s['Longitude']
                                  )

Scar_venues.head()

Malvern , Rouge
Rouge Hill , Port Union , Highland Creek
Guildwood , Morningside , West Hill
Woburn
Cedarbrae
Scarborough Village
Kennedy Park , Ionview , East Birchmount Park
Golden Mile , Clairlea , Oakridge
Cliffside , Cliffcrest , Scarborough Village West
Birch Cliff , Cliffside West
Dorset Park , Wexford Heights , Scarborough Town Centre
Wexford , Maryvale
Agincourt
Clarks Corners , Tam O'Shanter , Sullivan
Milliken , Agincourt North , Steeles East , L'Amoreaux East
Steeles West , L'Amoreaux West
Upper Rouge


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Malvern , Rouge",43.806686,-79.194353,Wendy’s,43.807448,-79.199056,Fast Food Restaurant
1,"Malvern , Rouge",43.806686,-79.194353,T Hamilton & Son Roofing Inc,43.807985,-79.198194,Construction & Landscaping
2,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,Royal Canadian Legion,43.782533,-79.163085,Bar
3,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,Scarborough Historical Society,43.788755,-79.162438,History Museum
4,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,SEBS Engineering Inc. (Sustainable Energy and ...,43.782371,-79.156820,Construction & Landscaping


In [13]:
# Let's check how many venues were returned for each neighborhood
Scar_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Birch Cliff , Cliffside West",4,4,4,4,4,4
Cedarbrae,8,8,8,8,8,8
"Clarks Corners , Tam O'Shanter , Sullivan",12,12,12,12,12,12
"Cliffside , Cliffcrest , Scarborough Village West",3,3,3,3,3,3
"Dorset Park , Wexford Heights , Scarborough Town Centre",6,6,6,6,6,6
"Golden Mile , Clairlea , Oakridge",8,8,8,8,8,8
"Guildwood , Morningside , West Hill",7,7,7,7,7,7
"Kennedy Park , Ionview , East Birchmount Park",4,4,4,4,4,4


In [14]:
## Analyze each neighborhood
# one hot encoding
Scar_onehot = pd.get_dummies(Scar_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Scar_onehot['Neighborhood'] = Scar_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Scar_onehot.columns[-1]] + list(Scar_onehot.columns[:-1])
Scar_onehot = Scar_onehot[fixed_columns]

Scar_onehot.head()

,Neighborhood,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,History Museum,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,"Malvern , Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Malvern , Rouge",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,"Rouge Hill , Port Union , Highland Creek",0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,"Rouge Hill , Port Union , Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,"Rouge Hill , Port Union , Highland Creek",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [15]:
# Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
Scar_grouped = Scar_onehot.groupby('Neighborhood').mean().reset_index()
Scar_grouped

,Neighborhood,American Restaurant,Asian Restaurant,Athletics & Sports,Auto Garage,Bakery,Bank,Bar,Breakfast Spot,Bubble Tea Shop,Bus Line,Bus Station,Café,Caribbean Restaurant,Chinese Restaurant,Clothing Store,Coffee Shop,College Stadium,Construction & Landscaping,Department Store,Discount Store,Electronics Store,Fast Food Restaurant,Fried Chicken Joint,Gas Station,General Entertainment,Grocery Store,Hakka Restaurant,History Museum,Ice Cream Shop,Indian Restaurant,Intersection,Italian Restaurant,Korean Restaurant,Latin American Restaurant,Light Rail Station,Lounge,Medical Center,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Motel,Noodle House,Park,Pet Store,Pharmacy,Pizza Place,Playground,Rental Car Location,Sandwich Place,Shopping Mall,Skating Rink,Soccer Field,Spa,Supermarket,Thai Restaurant,Vietnamese Restaurant
0,Agincourt,0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.00,0.000,0.00,0.000,0.000000,0.2,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.2,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.200000,0.000,0.0,0.000000,0.000000,0.000000
1,"Birch Cliff , Cliffside West",0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.25,0.000,0.000000,0.0,0.000000,0.25,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.25,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,0.000,0.0,0.000000,0.000000,0.000000
2,Cedarbrae,0.000000,0.00,0.125,0.000000,0.125000,0.125000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.125,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.125000,0.125000,0.00,0.000000,0.125,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.125000,0.000000
3,"Clarks Corners , Tam O'Shanter , Sullivan",0.000000,0.00,0.000,0.000000,0.000000,0.083333,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.000,0.083333,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.083333,0.083333,0.083333,0.00,0.000000,0.000,0.000000,0.000,0.000000,0.083333,0.083333,0.000000,0.0,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.083333,0.00,0.000000,0.083333,0.166667,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.083333,0.000000
4,"Cliffside , Cliffcrest , Scarborough Village West",0.333333,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000,0.000000,0.000000,0.333333,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.333333,0.000,0.0,0.000000,0.000000,0.000000
5,"Dorset Park , Wexford Heights , Scarborough To...",0.000000,0.00,0.000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000,0.00,0.000,0.166667,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.000,0.333333,0.000000,0.000000,0.000000,0.0,0.166667,0.0,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00,0.166667,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000,0.0,0.000000,0.000000,0.166667
6,"Golden Mile , Clairlea , Oakridge",0.000000,0.00,0.000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.25,0.125,0.00,0.000,0.000000,0.0,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.125,0.000000,0.000000,0.000000,0.000000,0.00

In [24]:
# For that what we want is to open a new pet store, so we should choose just some of the columns.
df_pet = Scar_grouped.loc[:,["Neighborhood","Café","Park","Clothing Store","Supermarket","Spa","Bank"]]

In [25]:
# Let's print each neighborhood along with the top 5 most common venues.
num_top_venues = 5

for hood in df_pet['Neighborhood']:
    print("----"+hood+"----")
    temp = df_pet[df_pet['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
            venue  freq
0  Clothing Store   0.2
1            Café   0.0
2            Park   0.0
3     Supermarket   0.0
4             Spa   0.0


----Birch Cliff , Cliffside West----
            venue  freq
0            Café  0.25
1            Park  0.00
2  Clothing Store  0.00
3     Supermarket  0.00
4             Spa  0.00


----Cedarbrae----
            venue  freq
0            Bank  0.12
1            Café  0.00
2            Park  0.00
3  Clothing Store  0.00
4     Supermarket  0.00


----Clarks Corners , Tam O'Shanter , Sullivan----
            venue  freq
0            Bank  0.08
1            Café  0.00
2            Park  0.00
3  Clothing Store  0.00
4     Supermarket  0.00


----Cliffside , Cliffcrest , Scarborough Village West----
            venue  freq
0            Café   0.0
1            Park   0.0
2  Clothing Store   0.0
3     Supermarket   0.0
4             Spa   0.0


----Dorset Park , Wexford Heights , Scarborough Town Centre----
            venue  freq
0

In [28]:
# Let's put that into a pandas dataframe
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = df_pet['Neighborhood']

for ind in np.arange(df_pet.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(df_pet.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(20)

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Clothing Store,Bank,Spa,Supermarket,Park
1,"Birch Cliff , Cliffside West",Café,Bank,Spa,Supermarket,Clothing Store
2,Cedarbrae,Bank,Spa,Supermarket,Clothing Store,Park
3,"Clarks Corners , Tam O'Shanter , Sullivan",Bank,Spa,Supermarket,Clothing Store,Park
4,"Cliffside , Cliffcrest , Scarborough Village West",Bank,Spa,Supermarket,Clothing Store,Park
5,"Dorset Park , Wexford Heights , Scarborough To...",Bank,Spa,Supermarket,Clothing Store,Park
6,"Golden Mile , Clairlea , Oakridge",Bank,Spa,Supermarket,Clothing Store,Park
7,"Guildwood , Morningside , West Hill",Bank,Spa,Supermarket,Clothing Store,Park
8,"Kennedy Park , Ionview , East Birchmount Park",Bank,Spa,Supermarket,Clothing Store,Park
9,"Malvern , Rouge",Bank,Spa,Supermarket,Clothing Store,Park


## 3. Now we could cluster those neighborhoods.

In [29]:
# set number of clusters
kclusters = 5

df_pet_clustering = df_pet.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(df_pet_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 1, 1, 1, 1, 1, 1, 1, 1])

In [30]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

pet_merged = df_s


# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
pet_merged = pet_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

pet_merged.head(20) 

,Postal code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353,1.0,Bank,Spa,Supermarket,Clothing Store,Park
1,M1C,Scarborough,"Rouge Hill , Port Union , Highland Creek",43.784535,-79.160497,1.0,Bank,Spa,Supermarket,Clothing Store,Park
2,M1E,Scarborough,"Guildwood , Morningside , West Hill",43.763573,-79.188711,1.0,Bank,Spa,Supermarket,Clothing Store,Park
3,M1G,Scarborough,Woburn,43.770992,-79.216917,1.0,Bank,Spa,Supermarket,Clothing Store,Park
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476,1.0,Bank,Spa,Supermarket,Clothing Store,Park
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476,2.0,Spa,Bank,Supermarket,Clothing Store,Park
6,M1K,Scarborough,"Kennedy Park , Ionview , East Birchmount Park",43.727929,-79.262029,1.0,Bank,Spa,Supermarket,Clothing Store,Park
7,M1L,Scarborough,"Golden Mile , Clairlea , Oakridge",43.711112,-79.284577,1.0,Bank,Spa,Supermarket,Clothing Store,Park
8,M1M,Scarborough,"Cliffside , Cliffcrest , Scarborough Village West",43.716316,-79.239476,1.0,Bank,Spa,Supermarket,Clothing Store,Park
9,M1N,Scarborough,"Birch Cliff , Cliffside West",43.692657,-79.264848,4.0,Café,Bank,Spa,Supermarket,Clothing Store


#### I don't know what happened with the last row. I finally decide to drop it.

In [31]:
pet_merged.drop(pet_merged.index[pet_merged['Neighborhood']=='Upper Rouge'].values,inplace=True)

In [44]:
pet_merged.head()
pet_merged.shape

(16, 11)

## 4. Good. Let's visulize the map.

In [32]:
# create map
latitude = pet_merged[["Latitude"]].mean()
longitude = pet_merged[["Longitude"]].mean()
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

s_labels=pet_merged['Cluster Labels'].astype("int")

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(pet_merged['Latitude'], pet_merged['Longitude'], pet_merged['Neighborhood'], s_labels):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## 5. Let's examine our clusters and decide which would be better for our visitor.

In [39]:
pet_merged.loc[pet_merged['Cluster Labels'] == 0, pet_merged.columns[[2] + list(range(5, pet_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
12,Agincourt,0.0,Clothing Store,Bank,Spa,Supermarket,Park


In [40]:
pet_merged.loc[pet_merged['Cluster Labels'] == 1, pet_merged.columns[[2] + list(range(5, pet_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,"Malvern , Rouge",1.0,Bank,Spa,Supermarket,Clothing Store,Park
1,"Rouge Hill , Port Union , Highland Creek",1.0,Bank,Spa,Supermarket,Clothing Store,Park
2,"Guildwood , Morningside , West Hill",1.0,Bank,Spa,Supermarket,Clothing Store,Park
3,Woburn,1.0,Bank,Spa,Supermarket,Clothing Store,Park
4,Cedarbrae,1.0,Bank,Spa,Supermarket,Clothing Store,Park
6,"Kennedy Park , Ionview , East Birchmount Park",1.0,Bank,Spa,Supermarket,Clothing Store,Park
7,"Golden Mile , Clairlea , Oakridge",1.0,Bank,Spa,Supermarket,Clothing Store,Park
8,"Cliffside , Cliffcrest , Scarborough Village West",1.0,Bank,Spa,Supermarket,Clothing Store,Park
10,"Dorset Park , Wexford Heights , Scarborough To...",1.0,Bank,Spa,Supermarket,Clothing Store,Park
11,"Wexford , Maryvale",1.0,Bank,Spa,Supermarket,Clothing Store,Park


In [41]:
pet_merged.loc[pet_merged['Cluster Labels'] == 2, pet_merged.columns[[2] + list(range(5, pet_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Scarborough Village,2.0,Spa,Bank,Supermarket,Clothing Store,Park


In [42]:
pet_merged.loc[pet_merged['Cluster Labels'] == 3, pet_merged.columns[[2] + list(range(5, pet_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
14,"Milliken , Agincourt North , Steeles East , L'...",3.0,Park,Bank,Spa,Supermarket,Clothing Store


In [43]:
pet_merged.loc[pet_merged['Cluster Labels'] == 4, pet_merged.columns[[2] + list(range(5, pet_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
9,"Birch Cliff , Cliffside West",4.0,Café,Bank,Spa,Supermarket,Clothing Store
